# Test 1

In this script, we will calculate similarity between 'question' and 'context' embedding values came from Vertex AI Gecko Embedding API. 

At fist, I will use 'textembedding-gecko@001'.

In [1]:
# ! pip install google-cloud-aiplatform
# ! pip install torch
# ! pip install transformers

In [2]:
import json
import datasets
from datasets import list_datasets

all_datasets = list_datasets()


/home/postgres/devel/rag_proto/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_18257/2335562021.py:5: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  all_datasets = list_datasets()


In [3]:
list = [x for x in all_datasets if 'quad' in x]
print(list)

# squad_kor_v2

['fquad', 'iapp_wiki_qa_squad', 'lc_quad', 'sberquad', 'squad', 'squad_adversarial', 'squad_es', 'squad_it', 'squad_kor_v1', 'squad_kor_v2', 'squad_v1_pt', 'squad_v2', 'squadshifts', 'thaiqa_squad', 'xquad', 'xquad_r', 'BSC-LT/viquiquad', 'BSC-LT/xquad-ca', 'GEM/squad_v2', 'KETI-AIR/korquad', 'Niciu/test-squad', 'Sabokou/qg_squad_modified', 'Sabokou/qg_squad_modified_dev', 'Tevatron/wikipedia-squad-corpus', 'Tevatron/wikipedia-squad', 'lmqg/qg_jaquad', 'lmqg/qg_squad', 'caltonji/harrypotter_squad_v2', 'caltonji/harrypotter_squad_v2_2', 'deepset/germanquad', 'dweb/squad_with_cola_scores', 'husnu/tquad-v1v2', 'husnu/tquad2', 'lhoestq/custom_squad', 'lijingxin/squad_zen', 'lijingxin/squad_zh_1', 'lincoln/newsquadfr', 'mmcquade11-test/reuters-for-summarization-two', 'philschmid/prompted-germanquad', 'philschmid/test_german_squad', 'phoelti/squad_dev', 'piEsposito/br-quad-2.0', 'piEsposito/br_quad_20', 'piEsposito/squad_20_ptbr', 'projecte-aina/vilaquad', 'projecte-aina/viquiquad', 'project

In [4]:
from datasets import load_dataset

squad_kor_v2 = load_dataset('squad_kor_v2')

In [5]:

validation_data = squad_kor_v2['validation']
df_valid = validation_data.to_pandas()

In [6]:
from google.auth import default

# Without Scopes, you will see the error. 
credentials, _ = default(scopes=["https://www.googleapis.com/auth/cloud-platform"])

In [7]:
import os

PROJECT_NUMBER = os.getenv("PROJECT_NUMBER")
VERTEX_AI_LOCATION = "us-central1"


In [8]:
import vertexai

vertexai.init(project=PROJECT_NUMBER, location=VERTEX_AI_LOCATION, credentials=credentials)
parameters = {
    "temperature" : 0.2,
    "max_output_tokens" : 1024, 
    "top_p": 0.8, 
    "top_k" : 10
}

In [9]:
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [10]:
df_embedding = df_valid[:200]

In [11]:
df_embedding['question']

0                             파스칼레 소틸레의 스파이크 높이는 몇 cm인가?
1      2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 201...
2      혈액 검사에서 나트륨의 정상범위는 최소 136mmol/L에서 최대 몇 mmol/L까...
3                    안토닌 드보르자크의 작품 중 18개의 가곡으로 이루어진 노래는?
4                 에드먼드 핼리가 달의 운동이 빨라지고 있다고 주장한 년도는 언제일까?
                             ...                        
195                                 유럽미래 협의회는 언제 만들어졌나요?
196              밀란과 리버풀의 2005 UEFE 챔피언스리그 결승전 성적은 어떠한가?
197                     2000호대 공항철도의 내부에 설치된 시트는 어떤 시트야?
198                  김우남이 제3회 지방선거의 제주도의원 선거에서 기록한 득표율은?
199    TBS의 한 프로그램에서 옴진리교 문제를 방영했다가 관련 문제의 인터뷰에 응했던 사...
Name: question, Length: 200, dtype: object

In [12]:
QUATA_LIMIT = 5

context_result_list = []
answer_result_list = []

for group_cnt in range(0, len(df_embedding) // QUATA_LIMIT):
  df_target = df_embedding[group_cnt * QUATA_LIMIT:(group_cnt + 1) * QUATA_LIMIT]
  context_result_list.extend(model.get_embeddings(df_target['context']))
  answer_result_list.extend(model.get_embeddings(df_target['question']))

In [13]:
len(context_result_list)

200

In [14]:
import pandas as pd

df_context_result = pd.DataFrame(context_result_list)
df_question_result = pd.DataFrame(answer_result_list)



In [15]:
df_question_result

,values,statistics,_prediction_response
0,"[-0.002634006552398205, 0.0392126701772213, 0....","{'token_count': 13.0, 'truncated': False}",([{'embeddings': {'values': [-0.00263400655239...
1,"[-0.04379591718316078, 0.04280420020222664, 0....","{'token_count': 33.0, 'truncated': False}",([{'embeddings': {'values': [-0.00263400655239...
2,"[-0.041056714951992035, 0.03452348709106445, -...","{'token_count': 30.0, 'truncated': False}",([{'embeddings': {'values': [-0.00263400655239...
3,"[-0.011794202961027622, 0.04595302790403366, 0...","{'token_count': 17.0, 'truncated': False}",([{'embeddings': {'values': [-0.00263400655239...
4,"[-0.014249068684875965, 0.0501880943775177, 0....","{'token_count': 19.0, 'truncated': False}",([{'embeddings': {'values': [-0.00263400655239...
...,...,...,...
195,"[-0.01334286481142044, 0.05682492256164551, 0....","{'token_count': 9.0, 'truncated': False}",([{'embeddings': {'values': [-0.01334286481142...
196,"[-0.030252812430262566, 0.04123174771666527, -...","{'token_count': 17.0, 'truncated': False}",([{'embeddings': {'values': [-0.01334286481142...
197,"[-0.025379586964845657, 0.05613899230957031, 0...","{'token_count': 15.0, 'truncated': False}",([{'embeddings': {'values': [-0.01334286481142...
198,"[0.013251530937850475, 0.036173656582832336, 0...","{'token_count': 17.0, 'truncated': False}",([{'embeddings': {'values': [-0.01334286481142...


In [16]:
import numpy as np

num_rows = len(df_context_result)
inner_product_matrix = np.zeros((num_rows, num_rows))

for i in range(num_rows):
  for j in range(num_rows):
    inner_product_matrix[i,j] = np.inner(df_context_result['values'][i], df_question_result['values'][j])


In [17]:
df_context_result['values']

0      [-0.03939630091190338, -0.005334157962352037, ...
1      [-0.03939630091190338, -0.005334157962352037, ...
2      [-0.017488885670900345, 0.007890645414590836, ...
3      [-0.02951676957309246, 0.0011924132704734802, ...
4      [-0.021410875022411346, -0.0009606317616999149...
                             ...                        
195    [-0.03291351720690727, -0.014041891321539879, ...
196    [-0.016701122745871544, 0.0007258627447299659,...
197    [-0.03628787025809288, 0.008938373997807503, 0...
198    [-0.01941458135843277, -0.013410494662821293, ...
199    [-0.012607574462890625, 0.007735342252999544, ...
Name: values, Length: 200, dtype: object

In [18]:
df_question_result['values']

0      [-0.002634006552398205, 0.0392126701772213, 0....
1      [-0.04379591718316078, 0.04280420020222664, 0....
2      [-0.041056714951992035, 0.03452348709106445, -...
3      [-0.011794202961027622, 0.04595302790403366, 0...
4      [-0.014249068684875965, 0.0501880943775177, 0....
                             ...                        
195    [-0.01334286481142044, 0.05682492256164551, 0....
196    [-0.030252812430262566, 0.04123174771666527, -...
197    [-0.025379586964845657, 0.05613899230957031, 0...
198    [0.013251530937850475, 0.036173656582832336, 0...
199    [-0.04408862441778183, 0.05550443008542061, -0...
Name: values, Length: 200, dtype: object

In [19]:
pd.DataFrame(inner_product_matrix)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.606955,0.617049,0.568916,0.563404,0.525711,0.525711,0.511562,0.517351,0.511428,0.511428,...,0.609771,0.529221,0.513555,0.512099,0.519944,0.512099,0.604928,0.599359,0.502472,0.581987
1,0.606955,0.617049,0.568916,0.563404,0.525711,0.525711,0.511562,0.517351,0.511428,0.511428,...,0.609771,0.529221,0.513555,0.512099,0.519944,0.512099,0.604928,0.599359,0.502472,0.581987
2,0.585668,0.548259,0.629075,0.531192,0.528521,0.528521,0.490056,0.519159,0.511675,0.511675,...,0.563100,0.543584,0.514859,0.505059,0.522546,0.505059,0.541044,0.554053,0.502195,0.583779
3,0.640734,0.608153,0.555539,0.585772,0.578018,0.578018,0.548373,0.566440,0.554744,0.554744,...,0.608218,0.598999,0.560212,0.551282,0.571123,0.551282,0.608155,0.610627,0.553750,0.590985
4,0.647143,0.624453,0.582655,0.592424,0.593544,0.593544,0.543560,0.581300,0.569531,0.569531,...,0.628152,0.610328,0.575482,0.560887,0.586135,0.560887,0.598908,0.633582,0.557950,0.599135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.630224,0.625407,0.582774,0.610326,0.570954,0.570954,0.550878,0.558090,0.547591,0.547591,...,0.654997,0.594352,0.551985,0.545735,0.563334,0.545735,0.658501,0.653928,0.538587,0.631667
196,0.613842,0.616137,0.533831,0.543822,0.550381,0.550381,0.521585,0.539560,0.529705,0.529705,...,0.637499,0.568597,0.534400,0.524491,0.544046,0.524491,0.666137,0.608065,0.539820,0.573021
197,0.604847,0.601305,0.573518,0.552650,0.540321,0.540321,0.505726,0.530980,0.522888,0.522888,...,0.614388,0.549363,0.526756,0.518307,0.534122,0.518307,0.584532,0.620191,0.511795,0.600306
198,0.633171,0.631823,0.567374,0.593747,0.568932,0.568932,0.537771,0.559071,0.549727,0.549727,...,0.643876,0.580815,0.553956,0.544159,0.562942,0.544159,0.625097,0.616605,0.543950,0.597732


In [20]:
# print(df_embedding['context'][0]) # 2016년 하계 올림픽 배구 남자 선수 명단 - 위키백과, 우리 모두의 백과사전
# print(df_embedding['context'][195]) # 니콜라스 베르그루엔 - 위키백과, 우리 모두의 백과사전
# print(df_embedding['question'][1]) # 2016년 하계 올림픽 캐나다 남자 배구 대표팀 선수들 중, 타일러 샌더스는 2015-16년에 어느 팀 소속이었나?

In [21]:
# In this dataset, there are duplicated context records which have different questions. 

list_same_context_question_product = []
list_different_context_question_product = []

for question_index in range(len(inner_product_matrix)):
  context_value = inner_product_matrix[question_index, question_index]
  list_same_context_question_product.append(context_value)
  for context_index in range(len(inner_product_matrix)):
    similarity = inner_product_matrix[context_index, question_index]
    if context_value == similarity:
      continue
    list_different_context_question_product.append(similarity)


In [22]:
## Right Context / Question pair similarity
pd.DataFrame(list_same_context_question_product).describe()

,0
count,200.000000
mean,0.580045
std,0.045118
min,0.497625
25%,0.544642
50%,0.574798
75%,0.605747
max,0.709523


In [23]:
pd.DataFrame(list_different_context_question_product).describe()

,0
count,39554.000000
mean,0.565062
std,0.033594
min,0.448161
25%,0.541504
50%,0.561378
75%,0.586798
max,0.709819


## Test 2.

New version Gecko Embedding API will be tested.

In [24]:

def make_embeddings(model, df_embedding):
  context_result_list = []
  question_result_list = []
  for group_cnt in range(0, len(df_embedding) // QUATA_LIMIT):
    df_target = df_embedding[group_cnt * QUATA_LIMIT:(group_cnt + 1) * QUATA_LIMIT]
    context_result_list.extend(model.get_embeddings(df_target['context']))
    question_result_list.extend(model.get_embeddings(df_target['question']))
  return context_result_list, question_result_list
    

In [25]:
new_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")

context_result_list_latest, question_result_list_latest = make_embeddings(new_model, df_embedding)

# 300 sec - 200 call 1.5 sec latency (per 5 batch)

In [26]:
df_context_latest = pd.DataFrame(context_result_list_latest)
df_question_latest = pd.DataFrame(question_result_list_latest)

In [27]:
def make_inner_product(df_context_result,df_question_result):
  num_rows = len(df_context_result)
  inner_product_matrix = np.zeros((num_rows, num_rows))
  for i in range(num_rows):
    for j in range(num_rows):
      inner_product_matrix[i,j] = np.inner(df_context_result['values'][i], df_question_result['values'][j])
  return inner_product_matrix

def make_diagonal_elements(inner_product_matrix):
  list_same_context_question_product = []
  list_different_context_question_product = []
  for question_index in range(len(inner_product_matrix)):
    context_value = inner_product_matrix[question_index, question_index]
    list_same_context_question_product.append(context_value)
    for context_index in range(len(inner_product_matrix)):
      similarity = inner_product_matrix[context_index, question_index]
      if context_value == similarity:
        continue
      list_different_context_question_product.append(similarity)
  return list_same_context_question_product, list_different_context_question_product

In [28]:
inner_product_matrix_latest = make_inner_product(df_context_latest, df_question_latest)
list_same_cq_sim_latest, list_dif_cq_sim_latest = make_diagonal_elements(inner_product_matrix_latest)

In [29]:
pd.DataFrame(list_same_cq_sim_latest).describe()

,0
count,200.000000
mean,0.698196
std,0.047039
min,0.593483
25%,0.662321
50%,0.699843
75%,0.728020
max,0.823053


In [30]:
pd.DataFrame(list_dif_cq_sim_latest).describe()

,0
count,39554.000000
mean,0.676202
std,0.038898
min,0.515430
25%,0.649293
50%,0.674321
75%,0.706634
max,0.813574


In [31]:
new_multilang_model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual@latest")

context_result_list_multilang, question_result_list_multilang = make_embeddings(new_multilang_model, df_embedding)


In [34]:
df_context_multilang = pd.DataFrame(context_result_list_multilang)
df_question_multilang = pd.DataFrame(question_result_list_multilang)

In [35]:
inner_product_matrix_multilang = make_inner_product(df_context_multilang, df_question_multilang)
list_same_cq_sim_multilang, list_dif_cq_sim_multilang = make_diagonal_elements(inner_product_matrix_multilang)

In [36]:
pd.DataFrame(list_same_cq_sim_multilang).describe()

,0
count,200.000000
mean,0.799890
std,0.044447
min,0.636418
25%,0.779767
50%,0.808998
75%,0.830088
max,0.887963


In [37]:
pd.DataFrame(list_dif_cq_sim_multilang).describe()

,0
count,39554.000000
mean,0.632010
std,0.040508
min,0.476688
25%,0.604111
50%,0.630727
75%,0.658613
max,0.813487
